# Feature engineering 

### Libraries

### Loading data

### Date-Related Features

### Time-Related Features

###  Rolling Window Feature

We could try computing the mean, the min, the max, or a weighted average where high weights would be affected to the most recent values.